### DataLoad

In [58]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
import pandas as pd
import numpy as np
from itertools import product
from scipy import sparse
from scipy.sparse import csr_matrix, lil_matrix
from sklearn.metrics.pairwise import cosine_similarity
import math
from tqdm import tqdm

test_interactions_path = '/content/drive/MyDrive/recsys project/steam_data/test_interactions.parquet'
train_interactions_path = '/content/drive/MyDrive/recsys project/steam_data/train_interactions.parquet'

test_interactions = pd.read_parquet(test_interactions_path)
train_interactions = pd.read_parquet(train_interactions_path)

In [72]:
train_interactions.describe()

,playerid,appid
count,9.117646e+06,9.117646e+06
mean,7.656120e+16,6.316143e+05
std,2.995367e+08,5.286173e+05
min,7.656120e+16,2.000000e+01
25%,7.656120e+16,2.890700e+05
50%,7.656120e+16,4.501900e+05
75%,7.656120e+16,7.823300e+05
max,7.656120e+16,3.339420e+06


### Data Preparation

In [73]:
def remove_top_active_players(train_df, test_df, percentile=95):
    """
      Remove the most active auditory since matrix operations were too big
    """

    player_game_counts = train_df['playerid'].value_counts()

    threshold = np.percentile(player_game_counts.values, percentile)

    players_to_keep = player_game_counts[player_game_counts <= threshold].index

    train_filtered = train_df[train_df['playerid'].isin(players_to_keep)].copy()
    test_filtered = test_df[test_df['playerid'].isin(players_to_keep)].copy()

    return train_filtered, test_filtered

train_95, test_95 = remove_top_active_players(train_interactions, test_interactions, percentile=95)


In [74]:
print(f"train: {len(train_interactions.index)}")
print(f"test: {len(test_interactions.index)}")

def sample_users(train_df, test_df, fraction=0.3, random_state=42):
    """
    Random sample of players from train
    """
    np.random.seed(random_state)

    all_players = train_df['playerid'].unique()
    sampled_players = np.random.choice(
        all_players,
        size=int(len(all_players) * fraction),
        replace=False
    )

    train_sampled = train_df[train_df['playerid'].isin(sampled_players)]
    test_sampled = test_df[test_df['playerid'].isin(sampled_players)]

    return train_sampled, test_sampled

train_sampled, test_sampled = sample_users(train_95, test_95, fraction=0.05)

print(f"Train: {len(train_sampled)}")
print(f"Test: {len(test_sampled)}")


train: 9117646
test: 44021
Train: 208328
Test: 2091


## Preparation

### Metrics function

In [64]:
def calculate_metrics(test_df, recommender_model, k=10, enable_tqdm=True):
    """
    Calculates HitRate@K, Recall@K, NDCG@K
    """
    known_users = set(recommender_model.user_map.keys())
    test_df_filtered = test_df[test_df['playerid'].isin(known_users)].copy()

    ground_truth = test_df_filtered.groupby('playerid')['appid'].apply(list).to_dict()

    hits = 0
    total_recall = 0
    total_ndcg = 0
    n_users = 0

    if len(ground_truth) == 0:
        return {"Error": "No overlapping users in test set"}

    for user, actual_items in tqdm(ground_truth.items(), disable= not (enable_tqdm)):
        try:
            recs = recommender_model.recommend(user, top_k=k)
            n_users += 1

            hit = any(item in actual_items for item in recs)
            if hit:
                hits += 1

            intersect = set(recs).intersection(set(actual_items))
            recall = len(intersect) / len(actual_items) if len(actual_items) > 0 else 0
            total_recall += recall

            dcg = 0
            idcg = 0

            for i, item in enumerate(recs):
                if item in actual_items:
                    dcg += 1 / math.log2((i + 1) + 1)

            num_relevant_in_top_k = min(len(actual_items), k)
            for i in range(num_relevant_in_top_k):
                idcg += 1 / math.log2((i + 1) + 1)

            ndcg = dcg / idcg if idcg > 0 else 0
            total_ndcg += ndcg
        except:
            continue

    return {
        f"HitRate@{k}": hits / n_users,
        f"Recall@{k}": total_recall / n_users,
        f"NDCG@{k}": total_ndcg / n_users
    }


### CF class

In [65]:
class CF:
    def __init__(self,
                 mode='user-based',
                 k_neighbors=50,
                 metric='cosine',
                 normalization='none',
                 threshold=0.0,
                 amplification=1.0,
                 verbose=True):
        """
        Collaborative filtering with all the bells and whistles
        """
        self.mode = mode
        self.k_neighbors = k_neighbors
        self.metric = metric
        self.normalization = normalization
        self.threshold = threshold
        self.amplification = amplification

        self.user_map = {}
        self.item_map = {}
        self.inv_user_map = {}
        self.inv_item_map = {}
        self.sparse_matrix = None
        self.normalized_matrix = None
        self.similarity_matrix = None
        self.user_means = None
        self.user_stds = None
        self.verbose = verbose

    def fit(self, df):
        """
        Build sparse matrix and precompute similarities for item-based mode
        """
        unique_users = df['playerid'].unique()
        unique_items = df['appid'].unique()

        self.user_map = {u: i for i, u in enumerate(unique_users)}
        self.item_map = {i: idx for idx, i in enumerate(unique_items)}
        self.inv_user_map = {i: u for u, i in self.user_map.items()}
        self.inv_item_map = {idx: i for i, idx in self.item_map.items()}

        user_indices = df['playerid'].map(self.user_map)
        item_indices = df['appid'].map(self.item_map)

        if 'rating' in df.columns:
            values = df['rating'].values
        else:
            values = np.ones(len(df))

        self.sparse_matrix = sparse.csr_matrix(
            (values, (user_indices, item_indices)),
            shape=(len(self.user_map), len(self.item_map))
        )

        self.normalized_matrix = self.normalize_matrix(self.sparse_matrix)

        if self.mode == 'item-based':
            if self.verbose: print("Precomputing itembased matrix")
            self.similarity_matrix = self.compute_similarity_matrix(
                self.normalized_matrix.T
            )
            if self.verbose: print(f"Sim matrix created: {self.similarity_matrix.shape}")

    def normalize_matrix(self, matrix):
        """
        Mean centering or z-score normalization if needed
        """
        if self.normalization == 'none':
            return matrix

        matrix = matrix.tocsr()

        if self.normalization == 'mean_center':
            self.user_means = np.array(matrix.mean(axis=1)).flatten()
            rows, cols = matrix.nonzero()
            data = matrix.data.copy()
            data -= self.user_means[rows]
            return sparse.csr_matrix((data, (rows, cols)), shape=matrix.shape)

        elif self.normalization == 'z_score':
            self.user_means = np.array(matrix.mean(axis=1)).flatten()
            self.user_stds = np.zeros(matrix.shape[0])
            for i in range(matrix.shape[0]):
                row_data = matrix.getrow(i).data
                if len(row_data) > 1:
                    self.user_stds[i] = row_data.std()
                else:
                    self.user_stds[i] = 1.0
            self.user_stds[self.user_stds == 0] = 1.0

            rows, cols = matrix.nonzero()
            data = matrix.data.copy()
            data = (data - self.user_means[rows]) / self.user_stds[rows]
            return sparse.csr_matrix((data, (rows, cols)), shape=matrix.shape)

        return matrix

    def compute_similarity_matrix(self, matrix):
        """
        Calculate similarity matrix with chosen metric
        """
        if self.metric == 'cosine':
            sim_matrix = cosine_similarity(matrix, dense_output=False)

        elif self.metric == 'pearson':
            sim_matrix = self.pearson_similarity(matrix)

        elif self.metric == 'adjusted_cosine':
            sim_matrix = self.adjusted_cosine_similarity(matrix)

        elif self.metric == 'jaccard':
            sim_matrix = self.jaccard_similarity(matrix)

        elif self.metric == 'dice':
            sim_matrix = self.dice_similarity(matrix)

        else:
            sim_matrix = cosine_similarity(matrix, dense_output=False)

        sim_matrix = self.apply_similarity_filters(sim_matrix, matrix)

        return sim_matrix

    def pearson_similarity(self, matrix):
        """
        Pearson correlation but optimized for sparse matrices
        """
        matrix = matrix.tocsr()
        n_rows = matrix.shape[0]

        row_means = np.array(matrix.sum(axis=1)).flatten() / np.diff(matrix.indptr)
        row_means = np.nan_to_num(row_means)

        rows, cols = matrix.nonzero()
        centered_data = matrix.data - row_means[rows]
        centered_matrix = csr_matrix((centered_data, (rows, cols)), shape=matrix.shape)

        squared_centered = centered_matrix.copy()
        squared_centered.data = squared_centered.data ** 2
        variances = np.array(squared_centered.sum(axis=1)).flatten() / np.diff(matrix.indptr)
        stds = np.sqrt(np.maximum(variances, 1e-10))

        rows, cols = centered_matrix.nonzero()
        normalized_data = centered_matrix.data / stds[rows]
        normalized_matrix = csr_matrix((normalized_data, (rows, cols)), shape=matrix.shape)

        sim_matrix = normalized_matrix @ normalized_matrix.T

        binary_matrix = (matrix != 0).astype(float)
        common_counts = binary_matrix @ binary_matrix.T
        common_counts = common_counts.toarray()

        sim_dense = sim_matrix.toarray()
        with np.errstate(divide='ignore', invalid='ignore'):
            sim_dense = sim_dense / np.maximum(common_counts - 1, 1)
        sim_dense = np.nan_to_num(sim_dense, nan=0.0)

        return csr_matrix(sim_dense)

    def jaccard_similarity(self, matrix):
        """
        Jaccard index for binary interactions
        """
        binary_matrix = (matrix != 0).astype(float).tocsr()

        intersection = binary_matrix @ binary_matrix.T

        row_sums = np.array(binary_matrix.sum(axis=1)).flatten()
        union = row_sums[:, np.newaxis] + row_sums[np.newaxis, :] - intersection.toarray()

        with np.errstate(divide='ignore', invalid='ignore'):
            jaccard = intersection.toarray() / np.maximum(union, 1)
        jaccard = np.nan_to_num(jaccard, nan=0.0)

        return csr_matrix(jaccard)

    def dice_similarity(self, matrix):
        """
        Dice coefficient, like Jaccard but more forgiving
        """
        binary_matrix = (matrix != 0).astype(float).tocsr()

        intersection = binary_matrix @ binary_matrix.T

        row_sums = np.array(binary_matrix.sum(axis=1)).flatten()
        sum_sizes = row_sums[:, np.newaxis] + row_sums[np.newaxis, :]

        with np.errstate(divide='ignore', invalid='ignore'):
            dice = 2 * intersection.toarray() / np.maximum(sum_sizes, 1)
        dice = np.nan_to_num(dice, nan=0.0)

        return csr_matrix(dice)

    def adjusted_cosine_similarity(self, matrix):
        """
        Cosine but adjusted by column means instead of row means
        """
        matrix = matrix.tocsr()

        matrix_t = matrix.T.tocsr()

        col_means = np.array(matrix_t.sum(axis=1)).flatten() / np.diff(matrix_t.indptr)
        col_means = np.nan_to_num(col_means)

        rows, cols = matrix.nonzero()
        centered_data = matrix.data - col_means[cols]
        centered_matrix = csr_matrix((centered_data, (rows, cols)), shape=matrix.shape)

        sim_matrix = cosine_similarity(centered_matrix, dense_output=False)

        return sim_matrix

    def apply_similarity_filters(self, sim_matrix, data_matrix):
        """
        Apply threshold and power amplification to similarities
        """
        sim_dense = sim_matrix.toarray()

        if self.amplification != 1.0:
            sim_dense = np.sign(sim_dense) * np.abs(sim_dense) ** self.amplification

        if self.threshold > 0:
            sim_dense[sim_dense < self.threshold] = 0

        return sparse.csr_matrix(sim_dense)

    def recommend(self, playerid, top_k=10):
        """
        Main recommendation function, returns top_k items for user
        """
        if playerid not in self.user_map:
            return []

        u_idx = self.user_map[playerid]
        user_vector = self.normalized_matrix[u_idx]
        original_vector = self.sparse_matrix[u_idx]

        if self.mode == 'user-based':
            scores = self.user_based_scores(u_idx, user_vector)
        else:
            scores = self._item_based_scores(user_vector)

        if self.normalization == 'mean_center':
            scores = scores + self.user_means[u_idx]
        elif self.normalization == 'z_score':
            scores = scores * self.user_stds[u_idx] + self.user_means[u_idx]

        if sparse.issparse(scores):
            scores = scores.toarray()
        scores = np.array(scores).flatten()

        scores[original_vector.indices] = -np.inf

        k_actual = min(top_k, len(scores))
        if k_actual == 0:
            return []

        top_indices = np.argpartition(scores, -k_actual)[-k_actual:]
        top_indices = top_indices[np.argsort(scores[top_indices])][::-1]

        recommendations = [
            self.inv_item_map[i] for i in top_indices
            if scores[i] > -np.inf
        ]

        return recommendations

    def user_based_scores(self, u_idx, user_vector):
        """
        User-based CF: find similar users and aggregate their ratings
        """
        if self.metric == 'cosine':
            user_similarities = cosine_similarity(
                user_vector,
                self.normalized_matrix
            ).flatten()
        else:
            sim_matrix = self.compute_similarity_matrix(self.normalized_matrix)
            user_similarities = sim_matrix[u_idx].toarray().flatten()

        user_similarities[u_idx] = -1

        if self.threshold > 0:
            user_similarities[user_similarities < self.threshold] = 0

        if self.amplification != 1.0:
            user_similarities = np.sign(user_similarities) * np.abs(user_similarities) ** self.amplification

        k_actual = min(self.k_neighbors, np.sum(user_similarities > 0))
        if k_actual <= 0:
            return np.zeros(self.normalized_matrix.shape[1])

        neighbor_indices = np.argpartition(user_similarities, -k_actual)[-k_actual:]
        neighbor_sims = user_similarities[neighbor_indices]
        neighbor_sims = np.maximum(neighbor_sims, 0)

        if neighbor_sims.sum() == 0:
            return np.zeros(self.normalized_matrix.shape[1])

        neighbor_ratings = self.normalized_matrix[neighbor_indices]
        scores = neighbor_ratings.T.dot(neighbor_sims) / (neighbor_sims.sum() + 1e-8)

        return scores

    def _item_based_scores(self, user_vector):
        """
        Item-based CF: use precomputed item similarities
        """
        interacted_items = user_vector.indices

        if len(interacted_items) == 0:
            return np.zeros(self.normalized_matrix.shape[1])

        item_sims = self.similarity_matrix[interacted_items].toarray()
        user_ratings = user_vector.data

        if len(user_ratings) > 0:
            weighted_sims = item_sims.T * user_ratings
            denom = np.abs(item_sims).sum(axis=0) + 1e-8
            scores = weighted_sims.sum(axis=1) / denom
        else:
            scores = item_sims.sum(axis=0) / (len(interacted_items) + 1e-8)

        return scores

    def get_similar_items(self, appid, top_k=10):
        """
        Find similar items to given appid (only for item-based mode)
        """
        if self.mode != 'item-based':
            return []

        if appid not in self.item_map:
            return []

        item_idx = self.item_map[appid]
        similarities = self.similarity_matrix[item_idx].toarray().flatten()
        similarities[item_idx] = -1

        k_actual = min(top_k, len(similarities) - 1)
        if k_actual <= 0:
            return []

        top_indices = np.argpartition(similarities, -k_actual)[-k_actual:]
        top_indices = top_indices[np.argsort(similarities[top_indices])][::-1]

        similar_items = [
            self.inv_item_map[i] for i in top_indices
            if similarities[i] > self.threshold
        ]

        return similar_items


## Training and inference
### Gridsearch

In [75]:
train_sampled_tiny, test_sampled_tiny = sample_users(train_95, test_95, fraction=0.05)

print(f"Train: {len(train_sampled_tiny)}")
print(f"Test: {len(test_sampled_tiny)}")

Train: 208328
Test: 2091


In [67]:
def grid_search_cf(train_df, test_df, param_grid, test_size=0.2, top_k=10, metric=''):
    metric += str(top_k)

    param_names = list(param_grid.keys())
    param_values = list(param_grid.values())
    param_combinations = list(product(*param_values))

    print(f"\ntotal combinations: {len(param_combinations)}\n")

    results = []
    best_score = -np.inf
    best_params = None

    for params_tuple in param_combinations:
        params = dict(zip(param_names, params_tuple))

        try:
            model = CF(verbose=False, **params)
            model.fit(train_df)

            metrics = calculate_metrics(test_df, model, k=top_k, enable_tqdm=True)

            if 'Error' in metrics:
                print(f'error: {metrics["Error"]}', params)
                continue

            result = {'params': params, **metrics}
            results.append(result)

            if metrics[metric] > best_score:
                best_score = metrics[metric]
                best_params = params

            print(f"\nParams: {params}")
            print('\n'.join([f'{k}:\t{v:.4f}' for k, v in metrics.items()]))

        except Exception as e:
            print(f'error: {str(e)}', params)
            continue
        # break

    results_df = pd.DataFrame(results)

    print(f"\nbest params:")
    print(best_params)
    print(f"best {metric}: {best_score:.4f}")

    return results_df, best_params, best_score

param_grid = {
    'mode': ['user-based', 'item-based'],
    'k_neighbors': [100, 300, 500],
    'metric': ['adjusted_cosine'],
    'normalization': ['mean_center'],
    'threshold': [0.0, 0.1],
    'amplification': [1.0],
}

# Запускаем grid search
# results_df, best_params, best_score = grid_search_cf(
#     train_df=train_sampled_tiny,
#     test_df=test_sampled_tiny,
#     param_grid=param_grid,
#     test_size=0.2,
#     top_k=10,
#     metric='HitRate@'  # или 'precision@k', 'recall@k', 'hit_rate'
# )


### Final evaluation

In [68]:
%%time
model_name = 'User based CF'

model = CF(
    mode='user-based',
    k_neighbors=100,
    metric='adjusted_cosine',
    normalization='mean_center',
    threshold=0.1,
    amplification=1.0
    )

model.fit(train_sampled_tiny)

result = calculate_metrics(test_sampled_tiny, model, k=10)
result['model'] = model_name

print('\n'.join([f'{k}:\t{v}' for k, v in result.items()]))


100%|██████████| 2091/2091 [13:59<00:00,  2.49it/s]

HitRate@10:	0.20277379244380678
Recall@10:	0.20277379244380678
NDCG@10:	0.12633644127692262
model:	User based CF
CPU times: user 11min 49s, sys: 1min 33s, total: 13min 23s
Wall time: 13min 59s


In [69]:
%%time
model_name = 'Item based CF'

model = CF(
    mode='item-based',
    metric='adjusted_cosine',
    normalization='mean_center',
    threshold=0.1,
    amplification=1.0
    )

model.fit(train_sampled_tiny)

result = calculate_metrics(test_sampled_tiny, model, k=10)
result['model'] = model_name

print('\n'.join([f'{k}:\t{v}' for k, v in result.items()]))


Precomputing itembased matrix
Sim matrix created: (13866, 13866)


100%|██████████| 2091/2091 [00:33<00:00, 61.71it/s] 

HitRate@10:	0.19320899091343854
Recall@10:	0.19320899091343854
NDCG@10:	0.12136266060587712
model:	Item based CF
CPU times: user 24.4 s, sys: 14.9 s, total: 39.3 s
Wall time: 40.7 s


| Model | HitRate@10 | Recall@10 | NDCG@10 | Wall Time | CPU Time (user) | CPU Time (sys) |
|-------|------------|-----------|---------|-----------|-----------------|----------------|
| User based CF | 0.2028 | 0.2028 | 0.1263 | 13min 59s | 11min 49s | 1min 33s |
| Item based CF | 0.1932 | 0.1932 | 0.1214 | 40.7s | 24.4s | 14.9s |


The overall dataset shows comparably low metrics, it seems like the task of predicting players choices is quite hard